In [1]:
import os
import pandas as pd
import nibabel as nib
import numpy as np

import warnings
warnings.filterwarnings("ignore")

from skimage.measure import block_reduce
import numpy as np
from concurrent.futures import ProcessPoolExecutor
import time
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import tensorly as tl

#Debugging import
import importlib
var = 'TensorDecisionTreeRegressorP' #the published version of code
package = importlib.import_module(var)
for name, value in package.__dict__.items():
    if not name.startswith("__"):
        globals()[name] = value

from TensorDecisionTreeRegressorP import *

import os
import nibabel as nib
import numpy as np
import matplotlib as plt
import pandas as pd
from sklearn.model_selection import train_test_split


# File path to the CSV file
csv_file = '/Users/zc56/Documents/CommenDesktop/RICE/MyProject/Bayes_Tensor_Tree/3D-images/ADNIData.csv'
df = pd.read_csv(csv_file)

# Remove rows where ADAS11_bl is missing (NaN)
#df_cleaned = df.dropna(subset=['ADAS11_bl'])
df_cleaned = df.dropna(subset=['MMSE_bl'])

# Extract the 'ADAS11_bl' column as the y variable
#y_variable = df_cleaned['ADAS11_bl'].values
y_variable = df_cleaned['MMSE_bl'].values

# Split the dataframe based on the DX_bl column values
cn_group = df_cleaned[df_cleaned['DX_bl'] == 'CN']
ad_group = df_cleaned[df_cleaned['DX_bl'] == 'AD']
lmci_group = df_cleaned[df_cleaned['DX_bl'] == 'LMCI']

# Display the counts for each group after removing NA
print(f"CN group size: {cn_group.shape[0]}")
print(f"AD group size: {ad_group.shape[0]}")
print(f"LMCI group size: {lmci_group.shape[0]}")

# Directory containing the 3D images
directory = '/Users/zc56/Documents/CommenDesktop/RICE/MyProject/Bayes_Tensor_Tree/3D-images/3D-Images/bl'

# Initialize dictionaries to hold the images and y values for each group
cn_images, ad_images, lmci_images = [], [], []
cn_y, ad_y, lmci_y = [], [], []

# Function to load the images based on PTID matching and append y values
def load_images_and_y(group, image_list, y_list):
    for _, row in group.iterrows():
        ptid = row['PTID']
        # Find the corresponding file based on PTID
        filename = f'{ptid}.nii.gz'
        file_path = os.path.join(directory, filename)
        
        if os.path.exists(file_path):
            # Load the NIfTI file
            img = nib.load(file_path)
            data = img.get_fdata()
            
            # Append the 3D image data and y value to the respective lists
            image_list.append(data)
            y_list.append(row['ADAS11_bl'])
        else:
            print(f"File {filename} not found.")

# Load images and y values for each group
load_images_and_y(cn_group, cn_images, cn_y)
load_images_and_y(ad_group, ad_images, ad_y)
load_images_and_y(lmci_group, lmci_images, lmci_y)

# Convert lists of 3D images and y values to NumPy arrays
if cn_images:
    cn_tensor = np.stack(cn_images, axis=0)
    cn_y = np.array(cn_y)
    print(f"CN 4D tensor shape: {cn_tensor.shape}")
    print(f"CN y shape: {cn_y.shape}")
else:
    print("No CN images loaded.")

if ad_images:
    ad_tensor = np.stack(ad_images, axis=0)
    ad_y = np.array(ad_y)
    print(f"AD 4D tensor shape: {ad_tensor.shape}")
    print(f"AD y shape: {ad_y.shape}")
else:
    print("No AD images loaded.")

if lmci_images:
    lmci_tensor = np.stack(lmci_images, axis=0)
    lmci_y = np.array(lmci_y)
    print(f"LMCI 4D tensor shape: {lmci_tensor.shape}")
    print(f"LMCI y shape: {lmci_y.shape}")
else:
    print("No LMCI images loaded.")

CN group size: 229
AD group size: 188
LMCI group size: 401
CN 4D tensor shape: (229, 48, 48, 48)
CN y shape: (229,)
AD 4D tensor shape: (188, 48, 48, 48)
AD y shape: (188,)
LMCI 4D tensor shape: (401, 48, 48, 48)
LMCI y shape: (401,)


In [35]:
X_train, X_test, y_train, y_test = train_test_split(lmci_tensor, lmci_y, test_size=0.2, random_state=42)


print(X_train.shape,y_train.shape)
model  =  TensorDecisionTreeRegressor(max_depth=6, min_samples_split=12,split_method='variance_LS', split_rank=4, CP_reg_rank=12, Tucker_reg_rank=12, n_mode=3)
model.use_mean_as_threshold  =  False
model.sample_rate  =  .5
model.prune()
X_coarsen_shape = (1,4,4,4)
X_coarsen_func = np.max
X_train_c = block_reduce(X_train,block_size=X_coarsen_shape, func=X_coarsen_func)
X_test_c = block_reduce(X_test,block_size=X_coarsen_shape, func=X_coarsen_func)
#middle_z = X_train_c.shape[2] // 2
#X_train_c = X_train_c[:,:,:,middle_z:middle_z+2]
#X_test_c = X_test_c[:,:,:,middle_z:middle_z+2]
X_train_c = X_train_c+np.ones_like(X_train_c)*1e-3
print(X_train_c.shape,y_train.shape,X_test_c.shape)
model.fit(X_train_c,y_train)


(320, 48, 48, 48) (320,)
(320, 12, 12, 12) (320,) (81, 12, 12, 12)


KeyboardInterrupt: 

In [30]:
predictions = model.predict(X_train_c,regression_method='mean')
print(f"mean train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train))
predictions = model.predict(X_train_c,regression_method='cp')
print(f"CP train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train))
predictions = model.predict(X_train_c,regression_method='tucker')
print(f"Tucker train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train)) 

predictions = model.predict(X_test_c,regression_method='mean')
print(f"mean test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))
predictions = model.predict(X_test_c,regression_method='cp')
print(f"CP test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))
predictions = model.predict(X_test_c,regression_method='tucker')
print(f"Tucker test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))

mean train RSE:  1.0
CP train RSE:  0.0007896630044508248
Tucker train RSE:  0.00024441466370847726
mean test RSE:  1.004552307627379
CP test RSE:  5.005207556792969
Tucker test RSE:  3.301803665448721


In [31]:
# Gradient Boosting Regressor Class
from GradientBoosting import *
# Initialize Gradient Boosting Regressor with a simple decision tree as weak learner
#weak_learner = TensorDecisionTreeRegressor(max_depth=3, min_samples_split=4, split_method='variance',split_rank=my_rank,n_mode=my_n_mode)
#weak_learner.use_mean_as_threshold = True
#weak_learner.sample_rate = 1.0
weak_learner = model
gradient_boosting_regressor = GradientBoostingRegressor(
    n_estimators=10,
    learning_rate=0.1,
    weak_learner=weak_learner
)
#gradient_boosting_regressor.pruning = True
# Fit a single tree model
weak_learner.fit(X_train_c, y_train)
weak_predictions = weak_learner.predict(X_test_c)


# Fit the Gradient Boosting model
gradient_boosting_regressor.fit(X_train_c, y_train, X_test_c, y_test)

fitting started
0 0 training RMSE === 1.0
0 0 testing RMSE === 1.004552307627379
0 1 training RMSE === 0.9469767409183073
0 1 testing RMSE === 1.0144315991814847
0 2 training RMSE === 1.0169600305498885
0 2 testing RMSE === 1.1739028262282187
0 3 training RMSE === 1.1782603484672332
0 3 testing RMSE === 1.4084120339887685
0 4 training RMSE === 1.393833457975776
0 4 testing RMSE === 1.7663138803345322
0 5 training RMSE === 1.6491414063994436
0 5 testing RMSE === 2.1293186061908105
0 6 training RMSE === 1.9275632058458
0 6 testing RMSE === 2.475761547617822
0 7 training RMSE === 2.2230824798618807
0 7 testing RMSE === 2.8662402328471366
0 8 training RMSE === 2.519918715058301
0 8 testing RMSE === 3.2353777572320896
0 9 training RMSE === 2.8151254360544122
0 9 testing RMSE === 3.6177170261012135


In [33]:
# Gradient Boosting Regressor Class
from GradientBoosting import *
# Initialize Gradient Boosting Regressor with a simple decision tree as weak learner
#weak_learner = TensorDecisionTreeRegressor(max_depth=3, min_samples_split=4, split_method='variance',split_rank=my_rank,n_mode=my_n_mode)
#weak_learner.use_mean_as_threshold = True
#weak_learner.sample_rate = 1.0
weak_learner = model
gradient_boosting_regressor = GradientBoostingRegressor(
    n_estimators=10,
    learning_rate=0.1,
    weak_learner=weak_learner
)
gradient_boosting_regressor.pruning = True
# Fit a single tree model
weak_learner.fit(X_train_c, y_train)
weak_predictions = weak_learner.predict(X_test_c)


# Fit the Gradient Boosting model
gradient_boosting_regressor.fit(X_train_c, y_train, X_test_c, y_test)

KeyboardInterrupt: 

In [ ]:
# Auto-slicing 
class AutoSlice:
    def __init__(self, n_slices = None, slicing_method = "cp", slicing_rank = 4, along_mode = 3):
        self.n_slices = n_slices
        self.slicing_method = slicing_method
        self.slicing_rank = slicing_rank
        self.along_mode = along_mode

    def _rank_k_approx_error(self, X):
        if len(X)<=self.min_samples_split:
            return np.inf
        if self.slicing_method=='cp':
            weights, factors = parafac(X, rank=self.slicing_rank, l2_reg = np.finfo(np.float32).eps)
            rank_k_approx = tl.cp_to_tensor((weights, factors))
            return tl.norm(X - rank_k_approx)
        if self.lowrank_method=='tucker':
            core, factors = tucker(X, rank=[X.shape[0],self.slicing_rank,self.split_rank])
            rank_k_approx = tl.tucker_to_tensor((core, factors)) 
            return tl.norm(X - rank_k_approx)
        if self.lowrank_method=='constrained_cp':
            weights, factors = constrained_parafac(X, rank=self.split_rank, l1_reg = 0.1) #To avoid super-sparse tensor
            rank_k_approx = tl.cp_to_tensor((weights, factors))
            return tl.norm(X - rank_k_approx)

    def _get_best_split(self, X, along_mode):
        if self.split_method == 'lowrank':
            error_type = 'low_rank'
            optimization_method = 'exhaustive'
            
        elif self.split_method == 'lowrank_LS':
            error_type = 'low_rank'
            optimization_method = 'LS'
            
        best_err = np.inf
        best_feature_index = None
        best_threshold = None

        # Define error function based on error_type, compatible with legacy options
        def error_function(subset):
            if error_type == 'low_rank':
                return self._rank_k_approx_error(subset)
            else:
                raise ValueError("Unsupported error type")

        # Define optimization strategy
        if optimization_method == 'exhaustive':
            #indices = np.ndindex(X.shape[1:])
            mode_indices = range(X.shape[along_mode])

        elif optimization_method == 'LS':
            #shape_of_interest = X.shape[1:]
            #total_combinations = np.prod(shape_of_interest)
            sample_size = max(1, int(self.sample_rate * range(X.shape[along_mode])))
            #b2variances = np.array([np.var(X[(slice(None),) + indices]) for indices in np.ndindex(shape_of_interest)])
            #print(variances)
            #p_vecs = variances / np.sum(variances)
            #print(p_vecs)
            sampled_indices = np.random.choice(range(X.shape[along_mode]), max(1, sample_size), replace=False)
            #indices = (np.unravel_index(idx, shape_of_interest) for idx in sampled_indices)
            mode_indices = sampled_indices
        else:
            raise ValueError(f"Unsupported optimization method: {optimization_method} or error function {error_type}.")

        # Loop over selected indices for the chosen optimization strategy
        if optimization_method in ['exhaustive', 'LS']:
            search_counter = 0
            for feature_index in mode_indices:
                search_counter = search_counter + 1
                if along_mode == 1:
                    #feature_values = X[:,feature_index,:,:]
                    candidate_idx_left = X[:,:feature_index,:,:]
                    candidate_idx_right = X[:,feature_index:,:,:]
                elif along_mode == 2:
                    #feature_values = X[:,:,feature_index,:]
                    candidate_idx_left = X[:,:,:feature_index,:]
                    candidate_idx_right = X[:,:,feature_index:,:]
                elif along_mode == 3:
                    #feature_values =  X[:,:,:, feature_index]
                    candidate_idx_left = X[:,:,:, feature_index]
                    candidate_idx_right = X[:,:,:, feature_index]
                cur_err = error_function(candidate_idx_left) + error_function(candidate_idx_right)
                    #print(optimization_method, cur_err,np.sum(candidate_idx_left),'/',np.sum(candidate_idx_right))
                    
                if cur_err < best_err:
                    best_err = cur_err
                    best_feature_index = feature_index
                    #best_threshold = threshold
                
                    
            #print('search size:',search_counter)
        #print(best_feature_index, best_threshold, best_err)
        return best_feature_index, best_err#best_threshold, best_err
    









    def _build_tree(self, X, y, depth=0):
        if len(y) >= self.min_samples_split:
            feature_index, threshold, loss = self._get_best_split(X)
            if feature_index is None:
                if self.n_mode == 3:
                    feature_index = (0,0)
                if self.n_mode == 4:
                    feature_index = (0,0,0)
            if threshold is None:
                if self.n_mode == 3:
                    threshold = X[0,0,0]
                if self.n_mode == 4:
                    threshold = X[0,0,0,0]
                print('None-->X',X.shape,X)
            #print('_build_tree',feature_index,threshold)
            node = Node(predicted_value=np.nan, samples_X=X, samples_y=y, leaf_index=self.leaf_counter)
            if (feature_index is not None) and (threshold is not None):
                #print('_build_tree:branch1')
                splits = self._split(X, y, feature_index, threshold)
                
                #print('build_tree=',splits)#splits is in the format of less_equal_than_threshold, greater_than_threshold
                self.leaf_counter = self.leaf_counter + 1
                node.root = self.root
                node.split_loss = loss
                node.feature_index = feature_index
                node.threshold = threshold
                node.left = self._build_tree(X[splits[0]], y[splits[0]], depth+1)
                if node.left is not None:
                    node.left.parent = node
                node.right = self._build_tree(X[splits[1]], y[splits[1]], depth+1)
                if node.right is not None:
                    node.right.parent = node
                #return node
            #else:
                #raise NotImplementedError('Unhandled Error: _build_tree (X/y shapes, depth)', X.shape, y.shape, depth,'feature_index', feature_index, 'threshold', threshold)
                # Train both CP and Tucker models at the leaf node
                #print('Xs',X.shape)
            cp_model = None
            tucker_model = None
            node.predicted_value = None
            if (node.left is None) and (node.right is None):
                node.predicted_value = np.mean(y)
                if self.adaptive_rank is not None and X.shape[0]>=self.adaptive_rank(depth,'cp'):
                    cp_model = CPRegressor(weight_rank=self.adaptive_rank(depth,'cp'),verbose=self.verbose)
                else:
                    cp_model = CPRegressor(weight_rank=self.CP_reg_rank,verbose=self.verbose)
                cp_model.fit(X, y)
                if self.adaptive_rank is not None:
                    tucker_model = TuckerRegressor(weight_ranks=self.adaptive_rank(depth,'tucker'), tol=10e-7, n_iter_max=self.modelmaxiter, reg_W=1, verbose=self.verbose)
                else:
                    tucker_model = TuckerRegressor(weight_ranks=self.Tucker_reg_rank, tol=10e-7, n_iter_max=self.modelmaxiter, reg_W=1, verbose=self.verbose)
                tucker_model.fit(X, y)
            self.leaf_counter = self.leaf_counter + 1
            node.cp_model=cp_model
            node.tucker_model=tucker_model
            node.leaf_index=self.leaf_counter
            return node
        else:
            #return None
            #print('_build_tree:branch2')
            # Train both CP and Tucker models at the leaf node
            if self.adaptive_rank is not None:
                cp_model = CPRegressor(weight_rank=self.adaptive_rank(depth,'cp'),verbose=self.verbose)
            else:
                cp_model = CPRegressor(weight_rank=self.CP_reg_rank,verbose=self.verbose)
            cp_model.fit(X, y)
            if self.adaptive_rank is not None:
                tucker_model = TuckerRegressor(weight_ranks=self.adaptive_rank(depth,'tucker'), tol=10e-7, n_iter_max=self.modelmaxiter, reg_W=1, verbose=self.verbose)
            else:
                tucker_model = TuckerRegressor(weight_ranks=self.Tucker_reg_rank, tol=10e-7, n_iter_max=self.modelmaxiter, reg_W=1, verbose=self.verbose)
            tucker_model.fit(X, y)
            
            #print(cp_model,tucker_model)
            self.leaf_counter = self.leaf_counter + 1
            return Node(predicted_value=np.mean(y), samples_X=X, samples_y=y, cp_model=cp_model, tucker_model=tucker_model, leaf_index=self.leaf_counter)

In [18]:
# Gradient Boosting Regressor Class
from GradientBoosting import *
# Initialize Gradient Boosting Regressor with a simple decision tree as weak learner
#weak_learner = TensorDecisionTreeRegressor(max_depth=3, min_samples_split=4, split_method='variance',split_rank=my_rank,n_mode=my_n_mode)
#weak_learner.use_mean_as_threshold = True
#weak_learner.sample_rate = 1.0
weak_learner = model
gradient_boosting_regressor = GradientBoostingRegressor(
    n_estimators=10,
    learning_rate=0.1,
    weak_learner=weak_learner
)
gradient_boosting_regressor.pruning = True
# Fit a single tree model
weak_learner.fit(X_train_c, y_train)
weak_predictions = weak_learner.predict(X_test_c)


# Fit the Gradient Boosting model
gradient_boosting_regressor.fit(X_train_c, y_train, X_test_c, y_test)

fitting started
0 0 training RMSE === 0.30254314619532646
0 0 testing RMSE === 1.7877948076518326
0 1 training RMSE === 0.37689478295955936
0 1 testing RMSE === 1.609581981130422
0 2 training RMSE === 0.5736726366440453
0 2 testing RMSE === 1.5937583351518485
0 3 training RMSE === 0.8601381897976808
0 3 testing RMSE === 1.7007621229278687
0 4 training RMSE === 1.210986030513143
0 4 testing RMSE === 1.899774416495884
0 5 training RMSE === 1.6068102836507643
0 5 testing RMSE === 2.16693945338644
0 6 training RMSE === 2.032874548002152
0 6 testing RMSE === 2.483934560105665


KeyboardInterrupt: 

In [ ]:
#first layer auto-slicing

class AutoSlicing:
    def __init__(self, n_slices, sliced_mode, submodel): #one naive submodel is mean(sub_y)
        self.n_slices = n_slices
        self.submodel = submodel
        self.sliced_mode = sliced_mode
    
    def split(X, y, sliced_mode):
        current_error = 0
        for i in range(X.shape[sliced_mode]):
            
            
